In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
from datasets import load_dataset
import torch

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [12]:
class CFG:
    model_name="debertav3base"
    learning_rate=1.5e-5
    weight_decay=0.02
    hidden_dropout_prob= 0.0 # 0.005
    attention_probs_dropout_prob=0.0 # 0.005
    num_train_epochs=3
    n_splits=4
    batch_size=12
    random_seed=42
    save_steps=100
    max_length=512
    ability = 'readability' #grammar, completeness

In [10]:
DATA_DIR = './data/'

In [13]:
# with open(DATA_DIR+'train/'+CFG.ability+'_train.json', "r") as f:    
#     data_train = json.load(f)
# with open(DATA_DIR+'test/'+CFG.ability+'_test.json', "r") as f:    
#     data_test = json.load(f)

data = load_dataset('json',data_files={'train':DATA_DIR+'train/'+CFG.ability+'_train.json','test':DATA_DIR+'test/'+CFG.ability+'_test.json'})
#test = load_dataset('json',data_files=DATA_DIR+'test/'+CFG.ability+'_test.json')

Found cached dataset json (/home/work/.cache/huggingface/datasets/json/default-23aeb5a3fdccffaf/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
train = data['train'].shuffle(seed=42).select(range(1000))
test = data['test'].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /home/work/.cache/huggingface/datasets/json/default-23aeb5a3fdccffaf/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-2327788a30368567.arrow
Loading cached shuffled indices for dataset at /home/work/.cache/huggingface/datasets/json/default-23aeb5a3fdccffaf/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-c9401b101a0b2926.arrow


In [8]:
train[0]

{'text': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."? [SEP] If the premise is true and the person was told by a gentleman that the described situation happened to him, it suggests that there is a possibility that the same thing could have happened to the speaker as well. However, it does not conclusively prove or entail that the same thing actually did happen to the speaker. Rather, it implies that the speaker\'s situation might be similar to the gentleman\'s based on the information they received.',
 'question': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell 

In [15]:
MODEL_NAME = "microsoft/deberta-v3-base"

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=1, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/work/.local/miniconda3/envs/yys/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special 

In [46]:
train_encoding = tokenizer(
    train['text'],
    return_tensors='pt',
    padding=True,
    truncation=True
)
test_encoding = tokenizer(
    test['text'],
    return_tensors='pt',
    padding=True,
    truncation=True
)

In [47]:
import torch
from torch.utils.data import Dataset


class ScoreDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encoding = encodings
        self.labels = labels

    def __getitem__(self, idx):
        data = {key: val[idx] for key, val in self.encoding.items()}
        data['labels'] = torch.tensor(self.labels[idx],dtype=torch.float32)

        return data

    def __len__(self):
        return len(self.labels)

In [48]:
train_set = ScoreDataset(train_encoding, train['readability'])
test_set = ScoreDataset(test_encoding, test['readability'])

In [ ]:
train_set[1]

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
device

device(type='cuda')

In [49]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
dummy = tokenizer(train['text'][0], return_tensors='pt')

model(**dummy)

SequenceClassifierOutput(loss=None, logits=tensor([[0.0432]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [51]:
from tqdm.notebook import tqdm
from datasets import load_metric

def train(epoch, model, dataloader, optimizer, device):
    model.to(device)

    m1 = load_metric('mse')

    for e in range(1, epoch+1):
        total_loss = 0.
        preds = []
        labels = []
        progress_bar = tqdm(dataloader, desc=f'TRAIN - EPOCH {e} |')
        for data in progress_bar:
            data = {k:v.to(device) for k, v in data.items()}
            output = model(**data)
            
            current_loss = output.loss
            total_loss += current_loss.item()  # .item()을 사용하여 스칼라 값을 얻음
            
            preds += list(output.logits.squeeze().detach().cpu().numpy())
            labels += list(data['labels'].detach().cpu().numpy())

            optimizer.zero_grad()
            current_loss.backward()
            optimizer.step()

            progress_bar.set_description(f'TRAIN - EPOCH {e} | current-loss: {current_loss:.4f}')
        
        rmse = m1.compute(predictions=preds, references=labels, squared=False)
        avg = total_loss / len(dataloader.dataset)

        print('='*64)
        print(f"TRAIN - EPOCH {e} | LOSS: {avg:.4f} RMSE: {rmse}")
        print('='*64)

In [23]:
from torch.optim import AdamW

# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

In [52]:
train(5, model, train_loader, optimizer, device)


TRAIN - EPOCH 1 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 1 | LOSS: 0.0322 RMSE: {'mse': 0.996620854595321}


TRAIN - EPOCH 2 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 2 | LOSS: 0.0292 RMSE: {'mse': 0.9494141741943081}


TRAIN - EPOCH 3 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 3 | LOSS: 0.0251 RMSE: {'mse': 0.8896607963716799}


TRAIN - EPOCH 4 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 4 | LOSS: 0.0174 RMSE: {'mse': 0.7434175833629258}


TRAIN - EPOCH 5 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 5 | LOSS: 0.0127 RMSE: {'mse': 0.6357503323154271}


In [53]:
from tqdm.notebook import tqdm
from datasets import load_metric

def evaluate(model, dataloader, device):
    model.to(device)

    m1 = load_metric('mse')

    total_loss = 0.
    preds = []
    labels = []
    progress_bar = tqdm(dataloader, desc=f'EVAL |')
    for data in progress_bar:
        data = {k:v.to(device) for k, v in data.items()}
        with torch.no_grad():
            output = model(**data)
        
        current_loss = output.loss
        total_loss += current_loss.item()  # .item()을 사용하여 스칼라 값을 얻음
        
        preds += list(output.logits.squeeze().detach().cpu().numpy())
        labels += list(data['labels'].detach().cpu().numpy())

        progress_bar.set_description(f'EVAL | current-loss: {current_loss:.4f}')
    
    rmse = m1.compute(predictions=preds, references=labels, squared=False)
    avg = total_loss / len(dataloader.dataset)

    print('='*64)
    print(f"EVAL | LOSS: {avg:.4f} RMSE: {rmse}")
    print('='*64)

In [54]:
evaluate(model, test_loader, device)


EVAL |:   0%|          | 0/32 [00:00<?, ?it/s]

EVAL | LOSS: 0.0322 RMSE: {'mse': 1.0024117614225598}
